Install Python to read the environment variables from a .env file

In [1]:
! pip install python-dotenv

  Obtaining dependency information for python-dotenv from https://files.pythonhosted.org/packages/6a/3e/b68c118422ec867fa7ab88444e1274aa40681c606d59ac27de5a5588f082/python_dotenv-1.0.1-py3-none-any.whl.metadata
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


Install Neo4j driver

In [2]:
! pip install neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.0/198.0 kB 4.0 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for pytz from https://files.pythonhosted.org/packages/9c/3d/a121f284241f08268b21359bd425f7d4825cffc5ac5cd0e1b3d82ffd2b10/pytz-2024.1-py2.py3-none-any.whl.metadata
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
  Created wheel for neo4j: filename=neo4j-5.18.0-py3-none-any.whl size=273863 sha256=339b6a3ada9110a78c2aa24699155f9264d3708c4d0d81ee817b70e7515e38bc
  Stored in directory: /Users/faaiz/Library/Caches/pip/wheels/54/5c/8d/646324601710739f27a4b46c615dc12a6149f193d895997857
Successfully built neo4j

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


Load `.env` variables

In [5]:
import os
from dotenv import load_dotenv
from neo4j import GraphDatabase

load_dotenv()
uri = os.getenv("NEO4J_URI")
user = os.getenv("NEO4J_USER")
password = os.getenv("NEO4J_PASSWORD")

Create driver and define `execute_query()` method

In [6]:
driver = GraphDatabase.driver(uri, auth=(user, password))
def execute_query(query):
    with driver.session() as session:
        results = session.run(query)
        return [record for record in results]

##### Q1: Given an artist, return tracks

In [8]:
q1_query = """
MATCH (track:Track)-[:COMPOSED_BY]->(artist:Artist {name: 'Nathaniel Bassey'})
RETURN track.name AS TrackName
"""
q1_results = execute_query(q1_query)
print("Q1 Results:", q1_results)

Q1 Results: [<Record TrackName='See What the Lord Has Done'>]


![](../docs/images/Q1.png)

##### Q2: Given a list of artists, return tracks sorted by the occurrence at album level

In [9]:
q2_query = """
MATCH (artist:Artist)<-[:COMPOSED_BY]-(track:Track)-[:PART_OF]->(album:Album)
WHERE artist.name IN ['Jenifer', 'Jessie J', 'Frank Morgan']
WITH album, COUNT(DISTINCT artist) AS artistsCount
ORDER BY artistsCount DESC
MATCH (album)-[:PART_OF]-(track:Track)
RETURN track.name AS TrackName, album.name AS Album ,artistsCount
"""
q2_results = execute_query(q2_query)
print("\nQ2 Results:", q2_results)


Q2 Results: [<Record TrackName="Nobody's Perfect - Tom Elmhirst Radio Edit" Album='Total Girl: Yearbook - Best Of 2011' artistsCount=2>, <Record TrackName='Sweet Talker' Album='Sugar Sweet' artistsCount=2>, <Record TrackName="Walkin'" Album='Reflections' artistsCount=1>, <Record TrackName='Jerry Rice' Album='Reflections' artistsCount=1>, <Record TrackName='Voices' Album='Reflections' artistsCount=1>, <Record TrackName='Frank and Poet' Album='Reflections' artistsCount=1>, <Record TrackName='Find A Reason' Album='Reflections' artistsCount=1>, <Record TrackName='Give It Up' Album='Reflections' artistsCount=1>]


![](../docs/images/Q2.png)

##### Q3: Given a set of tracks, return artists based on similarity at metadata level

In [11]:
q3_query = """
MATCH (track:Track)-[:HAS_GENRE]->(genre:Genre)<-[:HAS_GENRE]-(similarTrack:Track),
      (similarTrack)-[:COMPOSED_BY]->(artist:Artist)
WHERE track.name IN ['See What the Lord Has Done', 'Sing Your Blues Away']
RETURN DISTINCT artist.name AS ArtistName
"""
q3_results = execute_query(q3_query)
print("Q3 Results:", q3_results)

Q3 Results: [<Record ArtistName='The Daughters of Mary'>, <Record ArtistName=None>, <Record ArtistName='Nathaniel Bassey'>, <Record ArtistName='Todd Warren'>, <Record ArtistName='Enstrümantal'>, <Record ArtistName='Dr Tumi'>, <Record ArtistName='Mark Bishop'>, <Record ArtistName='Robert & Lea Sutanto'>, <Record ArtistName='Catherin Falla'>, <Record ArtistName='Alex Acheampong'>, <Record ArtistName='Cochren & Co.'>, <Record ArtistName='Jimmy Lahaie, Sebastian Demrey'>, <Record ArtistName='Ron Rawls'>, <Record ArtistName='Neptali Mendez'>, <Record ArtistName='Léa Mendonça'>, <Record ArtistName='Voz da Verdade'>, <Record ArtistName='Damaris Carbaugh'>, <Record ArtistName='GBENGA AKINFENWA'>, <Record ArtistName='Manuel Bonilla'>, <Record ArtistName='The Isbell Family'>, <Record ArtistName='La chorale Elisabeth Brasseur, Lucien Deiss, Maurice Fréchard, La Schola des Pères du Saint Esprit du Grand Scolasticat de Chevilly'>, <Record ArtistName='Dar Mjomba, Walter Chilambo'>, <Record ArtistNam

![](../docs/images/Q3.png)

##### Q4: Given a track return other tracks (from the same artist and/or similar artists that come from spotify_related_artists_ids)

This is a two step query. 
1. First we need to find the artist having more than one track 
2. For similar, we need to create another query


Q4.1: Find other tracks by the same artist

In [12]:
q4_1_results = """
MATCH (artist:Artist)<-[:COMPOSED_BY]-(track:Track)
WITH artist, COLLECT(track) AS tracks
WHERE SIZE(tracks) > 1
UNWIND tracks AS otherTrack
RETURN artist.name AS ArtistName, otherTrack.name AS TrackName

"""
q4_1_results = execute_query(q4_1_results)
print("Q Results:", q4_1_results)

Q Results: [<Record ArtistName='Dweezil Zappa' TrackName='Funky 15'>, <Record ArtistName='Dweezil Zappa' TrackName='Peaches En Regalia'>, <Record ArtistName='Matthias Kirschnereit' TrackName='Two Piano Pieces, WoO 54:: "Lustig & Traurig"'>, <Record ArtistName='Matthias Kirschnereit' TrackName='Violin Sonata No. 8 in G Major, Op. 30 No. 3: II. Tempo di minuetto, ma molto moderato e grazioso'>, <Record ArtistName='Matthias Kirschnereit' TrackName='Adagio; Allegro ("Les Adieux")'>, <Record ArtistName='Matthias Kirschnereit' TrackName='2. Larghetto'>, <Record ArtistName='Matthias Kirschnereit' TrackName='Violin Sonata No. 5, Op. 24: III. Scherzo. Allegro molto – Trio'>, <Record ArtistName='Matthias Kirschnereit' TrackName='Beethoven: 6 Bagatelles, Op. 126: No. 4 in B Major, Presto'>, <Record ArtistName='Matthias Kirschnereit' TrackName='Symphony No.6 in F, Op.68 -"Pastoral": 1. Erwachen heiterer Empfindungen bei der Ankunft auf dem Lande: Allegro ma non troppo'>, <Record ArtistName='Matthi

![](../docs/images/Q4_1.png)

Q4.2: Find tracks by artists similar based on shared genres

In [13]:
q4_2_results = """
MATCH (originalTrack:Track {name: 'Highway Junkie'})-[:HAS_GENRE]->(genre:Genre)<-[:HAS_GENRE]-(similarTrack:Track),
      (similarTrack)-[:COMPOSED_BY]->(similarArtist:Artist)
WHERE originalTrack.name <> similarTrack.name
RETURN similarArtist.name AS SimilarArtistName, COLLECT(similarTrack.name) AS SimilarTrackNames
"""
q4_2_results = execute_query(q4_2_results)
print("Q Results:", q4_2_results)

Q Results: [<Record SimilarArtistName=None SimilarTrackNames=["Couldn't Do Nothin' Right", "I Don't Owe You Nothing", 'Natural High', 'Pancho And Lefty', 'Pancho And Lefty', 'Conversation 16', 'Done Gone', 'Boogie', "I Don't Want Love", "I'm Alright", 'Whispering Hope', 'Peter Gunn', 'Peter Gunn', 'Angel of Light (feat. Myrkur)', "Propinquity (I've Just Begun to Care)", 'Eu Te Quero Demais', 'All Strung out Like Christmas Lights', 'Coffee Pot Blues', 'Born On Christmas Day', 'Neighbors', 'Vejdi po špičkách', 'Vejdi po špičkách', "Shakin' All Over", "Touch 'Em With Love", "Se Eu Pedir, Cê Volta - Live At Oliveira's Place/Goiânia(GO)-Brazil-2008", 'In God We Still Trust', 'Trucker Chicks', 'I Saw An Angel Die', 'Gone Away', 'Just the Way You Are', 'Honey Bee', 'Devil In Disguise', 'Ooh Las Vegas', "I Won't Forget You", 'What Are They Doing In Heaven Today', 'Yellow Creek', "This One's Gonna Hurt You (For A Long, Long Time)", "This One's Gonna Hurt You (For A Long, Long Time)", "You're Go

![](../docs/images/Q4_2.png)

## Q4_A Given a Track, find related Artists - 1st Level Traversal

In [14]:
q4_A_results = """
MATCH (track:Track {name: "Night Time Is the Right Time"})-[:COMPOSED_BY]->(artist:Artist)
WITH artist, track
MATCH (artist)-[:RELATED_TO]->(related:Artist)<-[:COMPOSED_BY]-(relatedTrack:Track)
RETURN track.name, artist.spotifyId AS ArtistSpotifyID, collect(related.spotifyId) AS SimilarArtistSpotifyIDs, collect(relatedTrack.name) AS SimilarTracks
"""
q4_A_results = execute_query(q4_A_results)
print("Q Results:", q4_A_results)

Q Results: [<Record track.name='Night Time Is the Right Time' ArtistSpotifyID='1eYhYunlNJlDoQhtYBvPsi' SimilarArtistSpotifyIDs=['4y6J8jwRAwO4dssiSmN91R', '4y6J8jwRAwO4dssiSmN91R', '7guDJrEfX3qb6FEbdPA5qi', '7guDJrEfX3qb6FEbdPA5qi', '7guDJrEfX3qb6FEbdPA5qi', '7guDJrEfX3qb6FEbdPA5qi', '7guDJrEfX3qb6FEbdPA5qi', '7guDJrEfX3qb6FEbdPA5qi', '7v4imS0moSyGdXyLgVTIV7', '7v4imS0moSyGdXyLgVTIV7', '7v4imS0moSyGdXyLgVTIV7', '7v4imS0moSyGdXyLgVTIV7', '5xLSa7l4IV1gsQfhAMvl0U', '5xLSa7l4IV1gsQfhAMvl0U', '5xLSa7l4IV1gsQfhAMvl0U', '5xLSa7l4IV1gsQfhAMvl0U', '0iOVhN3tnSvgDbcg25JoJb', '1YzCsTRb22dQkh9lghPIrp', '1YzCsTRb22dQkh9lghPIrp', '1YzCsTRb22dQkh9lghPIrp', '1YzCsTRb22dQkh9lghPIrp', '6hnWRPzGGKiapVX1UCdEAC', '7GaxyUddsPok8BuhxN6OUW', '19eLuQmk9aCobbVDHc6eek', '19eLuQmk9aCobbVDHc6eek', '19eLuQmk9aCobbVDHc6eek', '5V0MlUE1Bft0mbLlND7FJz', '5V0MlUE1Bft0mbLlND7FJz', '5V0MlUE1Bft0mbLlND7FJz', '5V0MlUE1Bft0mbLlND7FJz', '5V0MlUE1Bft0mbLlND7FJz', '5V0MlUE1Bft0mbLlND7FJz', '5V0MlUE1Bft0mbLlND7FJz', '5V0MlUE1Bft0m

## Q4_B Given a Track, find related Artists - Second Level Traversal

In [15]:
q4_B_results = """
MATCH (track:Track {name: "Night Time Is the Right Time"})-[:COMPOSED_BY]->(artist:Artist)
WITH artist, track
MATCH (artist)-[:RELATED_TO*2..2]->(related:Artist)<-[:COMPOSED_BY]-(relatedTrack:Track)
RETURN track.name, artist.spotifyId AS ArtistSpotifyID, collect(related.spotifyId) AS SimilarArtistSpotifyIDs, collect(relatedTrack.name) AS SimilarTracks
"""
q4_B_results = execute_query(q4_B_results)
print("Q Results:", q4_B_results)

Q Results: [<Record track.name='Night Time Is the Right Time' ArtistSpotifyID='1eYhYunlNJlDoQhtYBvPsi' SimilarArtistSpotifyIDs=['1YzCsTRb22dQkh9lghPIrp', '1YzCsTRb22dQkh9lghPIrp', '1YzCsTRb22dQkh9lghPIrp', '1YzCsTRb22dQkh9lghPIrp', '6hnWRPzGGKiapVX1UCdEAC', '1eYhYunlNJlDoQhtYBvPsi', '1eYhYunlNJlDoQhtYBvPsi', '2hGh5VOeeqimQFxqXvfCUf', '2hGh5VOeeqimQFxqXvfCUf', '2hGh5VOeeqimQFxqXvfCUf', '3dkbV4qihUeMsqN4vBGg93', '3dkbV4qihUeMsqN4vBGg93', '19eLuQmk9aCobbVDHc6eek', '19eLuQmk9aCobbVDHc6eek', '19eLuQmk9aCobbVDHc6eek', '3rxeQlsv0Sc2nyYaZ5W71T', '3rxeQlsv0Sc2nyYaZ5W71T', '3rxeQlsv0Sc2nyYaZ5W71T', '3rxeQlsv0Sc2nyYaZ5W71T', '0iOVhN3tnSvgDbcg25JoJb', '5V0MlUE1Bft0mbLlND7FJz', '5V0MlUE1Bft0mbLlND7FJz', '5V0MlUE1Bft0mbLlND7FJz', '5V0MlUE1Bft0mbLlND7FJz', '5V0MlUE1Bft0mbLlND7FJz', '5V0MlUE1Bft0mbLlND7FJz', '5V0MlUE1Bft0mbLlND7FJz', '5V0MlUE1Bft0mbLlND7FJz', '5V0MlUE1Bft0mbLlND7FJz', '5V0MlUE1Bft0mbLlND7FJz', '5V0MlUE1Bft0mbLlND7FJz', '5V0MlUE1Bft0mbLlND7FJz', '5V0MlUE1Bft0mbLlND7FJz', '5V0MlUE1Bft0m

Q5: Given a set of tracks, return radios

In [16]:
q5_query = """
MATCH (track:Track)-[:PLAYS]-(radio:Radio)
WHERE track.name IN ['No Goodbyes', 'Around the World in 80 Days']
RETURN DISTINCT radio.name AS RadioName

"""
q5_results = execute_query(q5_query)
print("Q Results:", q5_results)

Q Results: [<Record RadioName='Outlaw Country'>, <Record RadioName='WHOM 94.9 HOM'>]


![](../docs/images/Q5.png)

#### Q6: Given a Set of Artists, Return Tracks

In [17]:
q6_query = """
MATCH (artist:Artist)<-[:COMPOSED_BY]-(track:Track)
WHERE artist.name IN ["Matthias Kirschnereit", "Angela Yoffe & Vadim Gluzman"]
RETURN artist.name AS Artist, collect(track.name) AS Tracks
"""
q6_results = execute_query(q6_query)
print("Q Results:", q6_results)

Q Results: [<Record Artist='Matthias Kirschnereit' Tracks=['Two Piano Pieces, WoO 54:: "Lustig & Traurig"', 'Two Piano Pieces, WoO 54:: "Lustig & Traurig"', 'Two Piano Pieces, WoO 54:: "Lustig & Traurig"', 'Two Piano Pieces, WoO 54:: "Lustig & Traurig"', 'Two Piano Pieces, WoO 54:: "Lustig & Traurig"', 'Two Piano Pieces, WoO 54:: "Lustig & Traurig"', 'Two Piano Pieces, WoO 54:: "Lustig & Traurig"', 'Two Piano Pieces, WoO 54:: "Lustig & Traurig"', 'Two Piano Pieces, WoO 54:: "Lustig & Traurig"', 'Violin Sonata No. 8 in G Major, Op. 30 No. 3: II. Tempo di minuetto, ma molto moderato e grazioso', 'Violin Sonata No. 8 in G Major, Op. 30 No. 3: II. Tempo di minuetto, ma molto moderato e grazioso', 'Violin Sonata No. 8 in G Major, Op. 30 No. 3: II. Tempo di minuetto, ma molto moderato e grazioso', 'Violin Sonata No. 8 in G Major, Op. 30 No. 3: II. Tempo di minuetto, ma molto moderato e grazioso', 'Violin Sonata No. 8 in G Major, Op. 30 No. 3: II. Tempo di minuetto, ma molto moderato e grazio

#### Q7: Given a Set of Tracks, Return Artists (Use Artist Similarity 1st and/or 2nd Order)

In [18]:
q7_query = """
MATCH (track:Track)-[:COMPOSED_BY]->(artist:Artist)
WHERE track.name IN ["Violin Sonata No. 8 in G Major, Op. 30 No. 3: II. Tempo di minuetto, ma molto moderato e grazioso",  "Romanzen und Balladen II, Op. 49: I. Die beiden Grenadiere"] 
WITH artist
MATCH (artist)-[:RELATED_TO*1..2]->(similarArtist)
WHERE NOT artist = similarArtist
RETURN artist.name AS OriginalArtist, collect(DISTINCT similarArtist.name) AS SimilarArtists
"""
q7_results = execute_query(q7_query)
print("Q Results:", q7_results)

Q Results: [<Record OriginalArtist='Matthias Kirschnereit' SimilarArtists=['Geoffrey Parsons, Thomas Hampson, Richard Wagner', 'Johann Strauss II, Royal Concertgebouw Orchestra, Nikolaus Harnoncourt', 'Junko Okazaki', 'Angela Yoffe & Vadim Gluzman', 'Johann Sebastian Bach, Anonymous, Johannes Monno', 'Michel Piguet, Academy of Ancient Music, Christopher Hogwood, Antonio Vivaldi', 'The Angeles String Quartet, Franz Joseph Haydn', 'Giuseppe Verdi, Riccardo Chailly, Filarmonica della Scala', 'James Levine, Metropolitan Opera Orchestra, Wolfgang Amadeus Mozart, Bryn Terfel', 'Erik Satie, Aldo Ciccolini', 'The Eastman Chamber Ensemble, Hector Berlioz, David Effron, Jan DeGaetani', 'Gustav Mahler, New York Philharmonic, Leonard Bernstein', 'Carl Orff, Kölner Rundfunk Sinfonie Orchester, Tölzer Knabenchor, RIAS Kammerchor, Herbert von Karajan, Kölner Rundfunkchor', 'Wiener Philharmoniker;Andris Nelsons', 'Cody Quattlebaum', 'Maria Teresa Garatti, I Musici, Felix Ayo', 'Arthur Rubinstein', 'Va

#### Q8: Given a Set of Artists, Return Other Artists (Artist Similarity)

In [19]:
q8_query = """
MATCH (artist:Artist)-[:RELATED_TO]->(similarArtist)
WHERE artist.name IN ["Matthias Kirschnereit", "Angela Yoffe & Vadim Gluzman","Lorenzo Gatto, Ludwig van Beethoven, Julien Libeer","Angela Yoffe & Vadim Gluzman"] 
AND NOT similarArtist.name IN ["Matthias Kirschnereit", "Angela Yoffe & Vadim Gluzman","Lorenzo Gatto, Ludwig van Beethoven, Julien Libeer","Angela Yoffe & Vadim Gluzman"]
RETURN artist.name AS Artist, collect(DISTINCT similarArtist.name) AS SimilarArtists
"""
q8_results = execute_query(q8_query)
print("Q Results:", q8_results)

Q Results: [<Record Artist='Matthias Kirschnereit' SimilarArtists=['Erik Satie, Aldo Ciccolini', 'Michel Piguet, Academy of Ancient Music, Christopher Hogwood, Antonio Vivaldi', 'Giuseppe Verdi, Riccardo Chailly, Filarmonica della Scala', 'The Angeles String Quartet, Franz Joseph Haydn', 'Geoffrey Parsons, Thomas Hampson, Richard Wagner', 'James Levine, Metropolitan Opera Orchestra, Wolfgang Amadeus Mozart, Bryn Terfel', 'Johann Sebastian Bach, Anonymous, Johannes Monno', 'Junko Okazaki', 'Johann Strauss II, Royal Concertgebouw Orchestra, Nikolaus Harnoncourt']>, <Record Artist='Angela Yoffe & Vadim Gluzman' SimilarArtists=['Erik Satie, Aldo Ciccolini', 'The Angeles String Quartet, Franz Joseph Haydn', 'Johann Strauss II, Royal Concertgebouw Orchestra, Nikolaus Harnoncourt', 'Junko Okazaki', 'Johann Sebastian Bach, Anonymous, Johannes Monno', 'James Levine, Metropolitan Opera Orchestra, Wolfgang Amadeus Mozart, Bryn Terfel']>, <Record Artist='Lorenzo Gatto, Ludwig van Beethoven, Julien

In [ ]:

driver.close()

#### Additional Queries


##### To find other artists who have tracks in the same genres as a given set of tracks:

```sql
MATCH (track:Track)-[:HAS_GENRE]->(genre:Genre)<-[:HAS_GENRE]-(otherTrack:Track)
WHERE track.name IN ['Natural High', 'I Had a Dream'] AND track.name <> otherTrack.name
WITH DISTINCT genre, otherTrack
MATCH (otherTrack)-[:COMPOSED_BY]->(otherArtist:Artist)
RETURN genre.name AS GenreName, otherArtist.name AS OtherArtistName, COLLECT(otherTrack.name) AS OtherTracks
```

![](../docs/images/Q6.png)


##### To recommend tracks for a radio based on the genre:
```sql
MATCH (radio:Radio)-[:PLAYS]->(track:Track)-[:HAS_GENRE]->(genre:Genre)
WHERE radio.name = 'bigFM Balkan'
WITH genre
MATCH (genre)<-[:HAS_GENRE]-(recommendedTrack:Track)
RETURN genre.name AS GenreName, COLLECT(recommendedTrack.name) AS RecommendedTracks
```
![](../docs/images/Q7.png)

_Based on Album, Genre, there can be more....._